In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [14]:
## Load trained model, scaler and encoders
model = load_model('model.h5')

## encoders
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('OHE_geography.pkl', 'rb') as file:
    OHE_geography = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

In [15]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [16]:
geo_encoded = OHE_geography.transform([[input_data['Geography']]])
encoded_df = pd.DataFrame(geo_encoded,columns=OHE_geography.get_feature_names_out(['Geography']))
encoded_df

c:\Users\Chandiram\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [17]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [18]:
# Encode gender column
input_df['Gender'] = label_encoder_gender.transform([input_data['Gender']])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [19]:
## concat label and one hot encoding
input_df = pd.concat([input_df.drop('Geography',axis=1),encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
## scaling data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [21]:
## Prediction
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step


array([[0.05554007]], dtype=float32)

In [22]:
pred_prob = prediction[0][0]
pred_prob

0.05554007

In [23]:
if pred_prob > 0.5:
    print('The customer likely to churn')
else:
    print('The person likely not to churn')

The person likely not to churn
